In [1]:
!nvidia-smi

Mon Apr 17 21:51:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Установка соответствующих пакетов

In [2]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
!pip install progress==1.6
!python -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the 

## Подготовка данных

In [3]:
import torch
import re
import spacy

from progress.bar import ChargingBar 
from os.path import exists
from torch.utils.data import Dataset
from typing import List
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from timeit import default_timer as timer
from torch.nn.utils.rnn import pad_sequence

In [4]:
# Константные индексы особых токенов
BOS_IDX = 0 # begin of sentence
EOS_IDX = 1 # end of sentence
PAD_IDX = 2 # index for fulfilling
UNK_IDX = 3 # index for unknown token

data_path = '/content/'
reg_size = 346
com_size = 40000

train_size = 7000
valid_size = 3000

In [5]:
class MyDataset(Dataset):
    def __init__(self, list):
        self.data = list
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

In [6]:
def tokenize(text, tokenizer):
    return [tok.text for tok in tokenizer.tokenizer(text)]

def yield_tokens(data_iter, tokenizer, index):
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

def load_tokenizers():
    spacy_ru = spacy.load("ru_core_news_sm")
    return spacy_ru

spacy_ru = load_tokenizers()
    
def normString(str):
    str = str.lower().rstrip().lstrip()
    str = re.sub(r"([.!?])", r" \1", str)
    str = re.sub(r"[^а-яА-Я.!?]+", r" ", str)
    return str

def readTrainData(reg_data_path, com_data_path,
                  reg_proportion, com_proportion):
    lines = []
    
    com_train_size = round(com_size * com_proportion)
    if (com_train_size % 2 != 0):
        com_train_size -= 1
    with open(f"{com_data_path}new_corpus.txt", "r", encoding='cp1251') as f:
        for _ in range(0, com_train_size):
            line = f.readline()
            lines.append(line)
    
    data = [[normString(lines[i]), normString(lines[i+1])] for i in range(0, len(lines), 2)]
    return data

def readValidRegData(reg_data_path, reg_proportion):
    lines = []
    reg_valid_size = round(reg_size * reg_proportion)
    if (reg_valid_size % 2 != 0):
        reg_valid_size -= 1 

    miss_value = round(reg_size * (1 - reg_proportion))
    if (miss_value % 2 != 0):
        miss_value -= 1 

    with open(f"{reg_data_path}new_corpus.txt", "r", encoding='cp1251') as f:
        for _ in range(0, miss_value):
            line = f.readline()
        for _ in range(0, reg_valid_size):
            line = f.readline()
            lines.append(line)
    print(lines)
    data = [[normString(lines[i]), normString(lines[i+1])] for i in range(0, len(lines), 2)]
    return data

def readValidComData(com_data_path, com_proportion):
    lines = []
    com_valid_size = round(com_size * com_proportion)
    if (com_valid_size % 2 != 0):
        com_valid_size -= 1

    miss_value = round(reg_size * (1 - com_proportion))
    if (miss_value % 2 != 0):
        miss_value -= 1

    with open(f"{com_data_path}new_corpus.txt", "r", encoding='cp1251') as f:
        for _ in range(0, 40001):
            line = f.readline()
        for _ in range(40001, 48001):
            line = f.readline()
            lines.append(line)
    print(lines)
    data = [[normString(lines[i]), normString(lines[i+1])] for i in range(0, len(lines), 2)]
    return data
    

def tokenize_ru(text):
        return tokenize(text, spacy_ru)
    
def build_vocabulary(spacy_ru, train_data,
                     valid_data):

    print("Составление словаря посетителя ...")
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train_data + valid_data, tokenize_ru, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<pad>", "<unk>"],
    )
    print(type(vocab_src))

    print("Составление словаря регистратора ...")
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train_data + valid_data, tokenize_ru, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<pad>", "<unk>"],
    )
    print(type(vocab_tgt))

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt

def load_vocab(spacy_ru, train_data, valid_data):
    vocab_src, vocab_tgt = build_vocabulary(spacy_ru, train_data, valid_data)
    torch.save((vocab_src, vocab_tgt), "vocab.pt")
    
    print("Составление закончено.\nРазмеры словарей:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt

def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

def get_vocab_size(path):
    vocab_src, vocab_tgt = torch.load(path)
    return len(vocab_src), len(vocab_tgt)

In [7]:
train_data = readTrainData('/content/', '/content/', 0.8, 1)

valid_reg_data = readValidRegData('/content/', 0.2)
valid_com_data = readValidComData('/content/', 0.3)

vocab_src, vocab_trg = load_vocab(spacy_ru, train_data, 
                                  valid_reg_data + valid_com_data)

src_vocab_size = len(vocab_src.vocab)
trg_vocab_size = len(vocab_trg.vocab)    

train_iter = MyDataset(train_data)
valid_reg_iter = MyDataset(valid_reg_data)
valid_com_iter = MyDataset(valid_com_data)

text_transform_src = sequential_transforms(get_tokenizer('spacy', language='ru_core_news_sm'), 
                                               vocab_src, tensor_transform)

text_transform_trg = sequential_transforms(get_tokenizer('spacy', language='ru_core_news_sm'), 
                                               vocab_trg, tensor_transform)

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform_src(src_sample.rstrip("\n")))
        tgt_batch.append(text_transform_trg(tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

['Подтвердить не могу, но мне сказали, что номер будет забронирован.\n', 'Боюсь что брони на вас все-таки нет. Будем смотреть другие возможности?\n', 'Да, пожалуйста.\n', 'На какой период нужен номер?\n', 'Одна ночь.\n', 'Сколько человек?\n', 'Мне нужен номер на одного человека.\n', 'Подождите минутку. Присядьте.\n', 'Спасибо, я постою.\n', 'Вы торопитесь?\n', 'Нет, я никуда не спешу.\n', 'В каком ценовом сегменте рассматриваете номер?\n', 'Не имеет значения.\n', 'Какого размера кровать вы предпочитаете?\n', 'King (очень большой, 193 X 203 см)\n', 'Сейчас посмотрим, есть ли то, что вам нужно. Как настроение?\n', 'Спасибо, хорошее.\n', 'Ваш выбор, к сожалению, уже забронирован, будем смотреть другие варианты?\n', 'Да, я бы рассмотрел(а) другие варианты.\n', 'ОК, сейчас посмотрим. Как прошел ваш день?\n', 'День был прекрасный.\n', 'Вам нравится наш отель?\n', 'Да, очень.\n', 'Номер, который вас интересует, сейчас свободен. Могу дать со скидкой.\n', 'Хорошо, давайте продолжим оформление.\

In [8]:
print(len(train_data))
print(len(valid_reg_data))
print(len(valid_com_data))

20000
34
4000


## Нейронная сеть

In [9]:
import torch.nn as nn
import math

from torch import Tensor
from torch.nn import Transformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


In [11]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [12]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory, tgt_mask)

In [13]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


def translate(model: torch.nn.Module, src_sentence: str, vocab_trg, text_transform_src):
    model.eval()
    src = text_transform_src(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=128, start_symbol = BOS_IDX).flatten()
    return " ".join(vocab_trg.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<s>", "").replace("</s>", "")

## Обучение

In [14]:
!pip install gputil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, save, output_file

from GPUtil import showUtilization as gpu_usage
import pandas as pd

file_name = "loss_visualization.html"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
SRC_VOCAB_SIZE = len(vocab_src)
TGT_VOCAB_SIZE = len(vocab_trg)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 128
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6

NUM_EPOCHS = 20

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-8)

def save_nn_in_epoch(model, epoch):
    name = "Models/model_" + str(epoch) + "_epoch.pt"
    name_param = "Models/model_params_" + str(epoch) + "_epoch.pth"
    torch.save(model, name)
    torch.save(model.state_dict(), name_param)


def train_epoch(model, optimizer):
    model.train()
    losses = 0.0
    
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    bar = ChargingBar("Тренировка ...", max = len(train_dataloader))
    
    for src, tgt in train_dataloader:
        bar.next()
        src = src.type(torch.LongTensor)
        tgt = tgt.type(torch.LongTensor)
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        del src, tgt
    bar.finish()
    return losses / len(train_dataloader)


def evaluate_reg(model):
    model.eval()
    losses = 0.0

    valid_dataloader = DataLoader(valid_reg_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    bar = ChargingBar("Валидация на регистрационных диалогах...", max = len(valid_dataloader))
    for src, tgt in valid_dataloader:
        bar.next()
        src = src.type(torch.LongTensor)
        tgt = tgt.type(torch.LongTensor)
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
        del src, tgt
    bar.finish()
    return losses / len(valid_dataloader)

def evaluate_com(model):
    model.eval()
    losses = 0.0

    valid_dataloader = DataLoader(valid_com_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    bar = ChargingBar("Валидация на общих диалогах...", max = len(valid_dataloader))
    for src, tgt in valid_dataloader:
        bar.next()
        src = src.type(torch.LongTensor)
        tgt = tgt.type(torch.LongTensor)
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
        del src, tgt
    bar.finish()
    return losses / len(valid_dataloader)

def train():
    epochs = []
    train_l = []
    valid_reg_l = []
    valid_com_l = []
    for epoch in range(1, NUM_EPOCHS+1):
        start_time = timer()
        train_loss = train_epoch(transformer, optimizer)
        end_time = timer()
        
        #valid_reg_loss = evaluate_reg(transformer)
        valid_com_loss = evaluate_com(transformer)
        save_nn_in_epoch(transformer, epoch)
        
        epochs.append(epoch)
        train_l.append(train_loss)
        #valid_reg_l.append(valid_reg_loss)
        valid_com_l.append(valid_com_loss)
        #Valid loss (Reg): {valid_reg_loss:.3f},
        #Valid loss (Com): {valid_com_loss:.3f}, 
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, \
               "f"Epoch time = {(end_time - start_time):.3f}s"))
    df = pd.DataFrame({
        "Epochs": epochs,
        "Train_loss": train_l,
        #"Valid_reg_loss": valid_reg_l,
        "Valid_com_loss": valid_com_l,
    })
    source = ColumnDataSource(df)
    plot = figure(title = "Зависимость ошибки на тренировочной и валидационной выборках от эпохи",
                  max_width=750, height=500,
                  x_axis_label = "Эпоха",
                  y_axis_label = "Ошибка")
    plot.line(x= 'Epochs', y='Train_loss',
              color='green', alpha=0.8, legend="Train loss", line_width=2,
              source=source)
    # plot.line(x= 'Epochs', y='Valid_reg_loss',
    #           color='red', alpha=0.8, legend='Valid loss (Reg)', line_width=2,
    #           source=source)
    plot.line(x= 'Epochs', y='Valid_com_loss',
              color='blue', alpha=0.8, legend='Valid loss (Com)', line_width=2,
              source=source)
    output_file(filename = file_name)
    df.to_excel("train_stat_excel.xlsx", index=False)
    df.to_csv("train_stat_csv.csv", index=False, sep=";")
    save(plot)
    print("Нейронная сеть обучилась")



torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
torch.manual_seed(0)
train()

Epoch: 1, Train loss: 1.799,                Epoch time = 19.398s
Epoch: 2, Train loss: 0.094,                Epoch time = 19.857s
Epoch: 3, Train loss: 0.058,                Epoch time = 19.972s
Epoch: 4, Train loss: 0.049,                Epoch time = 20.403s
Epoch: 5, Train loss: 0.045,                Epoch time = 19.959s
Epoch: 6, Train loss: 0.042,                Epoch time = 20.056s
Epoch: 7, Train loss: 0.040,                Epoch time = 19.882s
Epoch: 8, Train loss: 0.038,                Epoch time = 20.169s
Epoch: 9, Train loss: 0.038,                Epoch time = 19.965s
Epoch: 10, Train loss: 0.036,                Epoch time = 20.025s
Epoch: 11, Train loss: 0.035,                Epoch time = 20.011s
Epoch: 12, Train loss: 0.035,                Epoch time = 20.015s
Epoch: 13, Train loss: 0.034,                Epoch time = 19.971s
Epoch: 14, Train loss: 0.034,                Epoch time = 19.976s
Epoch: 15, Train loss: 0.033,                Epoch time = 20.011s
Epoch: 16, Train lo

Нейронная сеть обучилась
